<center><p style = "background-color : gold; border-radius: 5px 5px; padding : 10px"><strong>
<span style="color:#000">Построение эталонного датасета</span>

<span style="color:#00c3ff">**Предварительный план**</span>

- <span style="color:yellow">**Пишем функцию JSON с разбивкой текстов на 5 10 и 20:**</span>
- <span style="color:yellow">**Анализируем в ручную и заполняем самостоятельно данные**</span>

In [1]:
# Импорт
import pdfplumber
import re
import json
from nltk.tokenize import sent_tokenize
import warnings
import logging
import pandas as pd

# Отключаем предупреждения PyPDF2
logging.getLogger("pdfminer").setLevel(logging.ERROR)
warnings.filterwarnings("ignore", message="CropBox missing from /Page, defaulting to MediaBox")

<span style="color:#00c3ff">**Предполагаемая разметка json**</span>

```json
{
    "fragment": "Изучаемый текст",
    "category" : "К какой категории звук (Природа, Люди, животные и т.п.)",
    "sound": "Сам звук",
    "substantive": "Существительное или иное основопологающее слово звука"
  },
```

<span style="color:yellow">**Пишем функцию JSON с разбивкой текстов на 5 10 и 20:**</span>

In [2]:
def pdf_to_json(pdf_path, output_json, sentences_per_fragment=5):
    # Извлечение текста из PDF
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text()
    
    # Очистка текста
    text = re.sub(r'\s+', ' ', text).strip()
    
    # Разбивка на предложения
    sentences = sent_tokenize(text, language='russian')
    
    # Группировка в фрагменты
    fragments = []
    for i in range(0, len(sentences), sentences_per_fragment):
        fragment_sentences = sentences[i:i + sentences_per_fragment]
        
        fragments.append({
            "fragment": fragment_sentences,  # Массив предложений
            "category": "",                   # К какой категории звук (Природа, Люди, животные и т.п.)
            "sound": "",                      # Сам звук
            "substantive": ""                # Существительное или иное основопологающее слово звука
        })
    
    # Форматирование JSON
    class CustomJSONEncoder(json.JSONEncoder):
        def encode(self, obj):
            json_str = super().encode(obj)
            # Добавляем переносы строк и отступы для массива предложений
            json_str = json_str.replace('", "', '",\n                               "')
            return json_str
    
    # Сохранение
    with open(output_json, "w", encoding="utf-8") as f:
        json.dump(fragments, f, cls=CustomJSONEncoder, ensure_ascii=False, indent=4)

<span style="color:#00c3ff">**Разбиваем на 5, 10, 20 предложений файл json**</span>

In [3]:
for count, name in [(5, "rr_5.json"), (10, "rr_10.json"), (20, "rr_20.json")]:
    pdf_to_json("rr.pdf", name, sentences_per_fragment=count)

<span style="color:#00c3ff">**Проверяем полученный результат:**</span>

In [4]:
# Читаем датасет в виде json
df = pd.read_json('rr_10.json')
df

,fragment,category,sound,substantive
0,"[Эрих Мария Ремарк Жизнь взаймыЖизнь взаймы., ...",,,
1,"[Над телефонными столбами каркали вороны, а в ...",,,
2,"[Клерфэ заказал мясо по-швейцарски, порцию ваш...",,,
3,"[— подумал Клерфэ., — Кататься на лыжах, пожал...",,,
4,"[Почему вы со своим рысаком не на автостраде?,...",,,
...,...,...,...,...
708,"[— Хольман!, — крикнула Лилиан., — Ведь это Хо...",,,
709,"[— Что случилось, Хольман?, — спросила Лилиан....",,,
710,"[— Наша старая фирма., Вчера они мне позвонили...",,,
711,"[Он займет место Клерфэ, так же как когда-то К...",,,


<span style="color:yellow">**Анализируем в ручную и заполняем самостоятельно данные**</span>